In [1]:
from pyspark.sql import SparkSession

spark = (SparkSession
            .builder
            .master('local[1]')
            .appName('F1-Racing-DE')
            .getOrCreate())

23/05/09 21:20:07 WARN Utils: Your hostname, MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.6 instead (on interface en0)
23/05/09 21:20:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/09 21:20:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/09 21:20:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/09 21:20:07 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


### Race Results

In [11]:
from pyspark.sql.functions import col

In [24]:
races_df = spark.read.format('parquet').load('data/processed/races')
circuits_df = spark.read.format('parquet').load('data/processed/circuits')
drivers_df = spark.read.format('parquet').load('data/processed/drivers')
constructors_df = spark.read.format('parquet').load('data/processed/constructors')
results_df = spark.read.format('parquet').load('data/processed/results')

In [25]:
races_final_df = races_df.select(
    col('race_id'),
    col('circuit_id'),
    col('race_year'), 
    col('name').alias('race_name'), 
    col('race_timestamp').alias('race_date')
)

circuits_final_df = circuits_df.select(
    col('circuit_id'),
    col('location').alias('circuit_location')
)

drivers_final_df = drivers_df.select(
    col('driver_id'),
    col('number').alias('driver_number'),
    col('name').alias('driver_name'),
    col('nationality').alias('driver_nationality')
)

constructors_final_df = constructors_df.select(
    col('constructor_id'),
    col('name').alias('team')
)

results_final_df = results_df.select(
    col('result_id'),
    col('race_id'),
    col('driver_id'),
    col('constructor_id'),
    col('time').alias('race_time')
)

In [26]:
race_circuits_df = races_final_df.join(
    circuits_final_df,
    races_final_df.circuit_id == circuits_final_df.circuit_id,
    how='inner'
)

In [27]:
race_circuits_df.show()

+-------+----------+---------+--------------------+-------------------+----------+----------------+
|race_id|circuit_id|race_year|           race_name|          race_date|circuit_id|circuit_location|
+-------+----------+---------+--------------------+-------------------+----------+----------------+
|   1098|         3|     2023|  Bahrain Grand Prix|2023-03-05 15:00:00|         3|          Sakhir|
|   1099|        77|     2023|Saudi Arabian Gra...|2023-03-19 17:00:00|        77|          Jeddah|
|   1100|         1|     2023|Australian Grand ...|2023-04-02 05:00:00|         1|       Melbourne|
|   1101|        73|     2023|Azerbaijan Grand ...|2023-04-30 11:00:00|        73|            Baku|
|   1102|        79|     2023|    Miami Grand Prix|2023-05-07 19:30:00|        79|           Miami|
|   1103|        21|     2023|Emilia Romagna Gr...|2023-05-21 13:00:00|        21|           Imola|
|   1104|         6|     2023|   Monaco Grand Prix|2023-05-28 13:00:00|         6|     Monte-Carlo|
